# Imports

In [1]:
import json

from algosdk.v2client import algod

from setup.create_app import create_app
from setup.opt_in_app import opt_in_app
from setup.call_app import call_app
from setup.helper_functions import *

from setup.opt_in_to_asset.main import opt_into_asset



# Initializing Algod Client

In [2]:
# user declared algod connection parameters. Node must have EnableDeveloperAPI set to true in its config
algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

# initialize an algodClient
algod_client = algod.AlgodClient(algod_token, algod_address)

# Obtaining Private and Public Keys from keys.json

In [3]:
# open json file with address and mnemonic data
f = open('keys.json')
data = json.load(f)



# Creator address and private key. 
# This account is a multisig (1/2). 
# For developement purposes, both private keys are stored in keys.json
# This will change in a production environment for security purposes. 
creator_mnemonic_1 = data['creator_multisig_accounts']['creator_mnemonic_1']
creator_private_key_1 = get_private_key_from_mnemonic(creator_mnemonic_1)
creator_address = data['creator_multisig_address']


# Lender USDC Deposit Pool.
# This account is single sig Algorand account.
# The private key and address is stored in keys.json for development purposes
# This account will be rekeyed to the smart contract
lender_usdc_pool_mnemonic = data['lender_usdc_pool_mnemonic']
lender_usdc_pool_private_key = get_private_key_from_mnemonic(lender_usdc_pool_mnemonic)
lender_usdc_pool_address = data['lender_usdc_pool_address']



# User private key. 
# In this case it is a singlesig, but it could also be a multisig if the user wants to make it one. 
# For developement purposes, this private key is stored in keys.json
user_mnemonic = data['user_mnemonic']
user_private_key = get_private_key_from_mnemonic(user_mnemonic)
user_address = data['user_mnemonic']

# USDC ID
usdc_id = int(data['usdc_id'])

# Closing json file
f.close()

# Creating Application

In [4]:
app_id, application_address = create_app(algod_client, creator_address, creator_private_key_1)
print("Created new app-id:", app_id)
print("Application address:", application_address)

# read global state of application
print( "Global state:", read_global_state(algod_client, creator_address, app_id) )

# read local state of application from user account
print( "Local state:", read_local_state(algod_client, creator_address, app_id) )

Waiting for confirmation...
Transaction FRNR3OOTH7JGTO3GPHXILAYAYLREOARWESLIM74ZBYYFFP4VKCXQ confirmed in round 18770082.
Created new app-id: 55012959
Application address: SIID3S6XT376KDWO6TY7ZC7HJJZMSKHKTTXWXGW3A6TB67Z4UEKWEGLH3E
Global state: No global state at this time.
Local state: {}


##### --------------------------------------------------------------------------------------------
# **lender_usdc_pool**: 3 actions

##### 1) ----------> Opt into smart contract 
##### 2) ----------> Opt into USDC 
##### 3) ----------> Rekey to smart contract

##### --------------------------------------------------------------------------------------------


##### Action 1: Opt into Smart Contract

In [5]:
# if pool account is not opted into the app, opt into it
if not is_opted_in_app(algod_client, lender_usdc_pool_address, app_id):

    # opt into application from pool account
    result = opt_in_app(algod_client, lender_usdc_pool_private_key, app_id)

    # print result confirmation transaction confirmation 
    print_confirmation(result)
    

OptIn from account:  AWISZC74CIL7FYO6LFQUTVW2PCMXHQDQ7IUUGMQHJBMUZMNUR4FOJJ2YOI
Waiting for confirmation...
Transaction G736HOBB7HP74CRJA2YEJIR4VBD5VI3PZNXB7DS2WCQX2XA7QRMQ confirmed in round 18770084.
OptIn to app-id: 55012959
Result confirmed in round: 18770084


##### Action 2: Opt into USDC

In [4]:
# if pool is not opted into the asset, opt into it
if not is_opted_in_asset(algod_client, usdc_id, lender_usdc_pool_address):

    print("Account is not opted in to USDC.. opting in now...")

    # opt into USDC from pool account
    result = opt_into_asset(algod_client, lender_usdc_pool_address, lender_usdc_pool_private_key, usdc_id)

    # print result confirmation transaction confirmation 
    print_confirmation(result)

# print pool asset holdings
print_asset_holding(algod_client, lender_usdc_pool_address, usdc_id)


Asset ID: 10458941
{
    "amount": 17782761,
    "asset-id": 10458941,
    "creator": "VETIGP3I6RCUVLVYNDW5UA2OJMXB5WP6L6HJ3RWO2R37GP4AVETICXC55I",
    "is-frozen": false
}


##### Action 3: Rekey lender_usdc_pool to Smart Contract

# User OptIn to Application

In [5]:
# if account is not opted into the app, opt in
if not is_opted_in_app(algod_client, user_address, app_id):

    # opt into application from user account
    result = opt_in_app(algod_client, user_private_key, app_id)

    # print result confirmation transaction confirmation 
    print_confirmation(result)

    # print logs
    print_logs(result)


# read global state of application
print( "Global state:", read_global_state( algod_client, creator_address, app_id) )

# read local state of application from user account
print( "Local state:", read_local_state( algod_client, user_address, app_id) )

# Calling Application

In [ ]:
args = [b"vote", b"choiceA"]

if not is_opted_in_asset: 
    # opt in to asset function 
    pass
    
result = call_app(algod_client, user_private_key, app_id, args)    


# read local state of application from user account
print( "Local state:", read_local_state(algod_client, user_address, app_id) )

# read global state of application
print( "Global state:", read_global_state(algod_client, creator_address, app_id) )